In [86]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [87]:
joinExpresion = person['graduate_program'] == graduateProgram['id']

In [88]:
joinType = 'inner'

In [89]:
person.join(graduateProgram,joinExpresion,joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [90]:
graduateProgram2 = graduateProgram.union(spark.createDataFrame([
    (0,'Master',"Duplicated Row","Duplicated School")
]))
graduateProgram2.createOrReplaceTempView('graduateProgram2')

In [91]:
# graduateProgram2 내용 확인하기
graduateProgram2.show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  2|Masters|                EECS|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0| Master|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [92]:
spark.sql("""
    select * from graduateProgram2
""").show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  2|Masters|                EECS|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0| Master|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [93]:
sparkStatus.show(2)

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
+---+--------------+
only showing top 2 rows



In [94]:
from pyspark.sql.functions import expr
person.withColumnRenamed('id','persionid').join(sparkStatus,
                                                expr("array_contains(spark_status,id)")).show()

+---------+----------------+----------------+---------------+---+--------------+
|persionid|            name|graduate_program|   spark_status| id|        status|
+---------+----------------+----------------+---------------+---+--------------+
|        0|   Bill Chambers|               0|          [100]|100|   Contributor|
|        1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|        1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|        1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|        2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|        2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+---------+----------------+----------------+---------------+---+--------------+



In [95]:
# 2010-summary.csv
# .option('mode','FAILEFAST') 잘못된 레코드를 발견하면 즉시 반환
csvFile = spark.read.format('csv').option('header','true').option('inferSchema','true')\
.option('mode','FAILFAST').load('2015-summary.csv')

In [96]:
csvFile.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [97]:
csvFile.write.format('csv').mode('overwrite').option('sep','\t')\
.save('my-file.tsv')

In [98]:
csvFile2 = spark.read.format('csv').option('header','true').option('inferSchema','true')\
.option('mode','FAILFAST').option('sep','\t').load('my-file.tsv')

In [99]:
csvFile2.show()

+--------------------+----------------+---+
|       United States|         Romania| 15|
+--------------------+----------------+---+
|       United States|         Croatia|  1|
|       United States|         Ireland|344|
|               Egypt|   United States| 15|
|       United States|           India| 62|
|       United States|       Singapore|  1|
|       United States|         Grenada| 62|
|          Costa Rica|   United States|588|
|             Senegal|   United States| 40|
|             Moldova|   United States|  1|
|       United States|    Sint Maarten|325|
|       United States|Marshall Islands| 39|
|              Guyana|   United States| 64|
|               Malta|   United States|  1|
|            Anguilla|   United States| 41|
|             Bolivia|   United States| 30|
|       United States|        Paraguay|  6|
|             Algeria|   United States|  4|
|Turks and Caicos ...|   United States|230|
|       United States|       Gibraltar|  1|
|Saint Vincent and...|   United 

In [100]:
# json read show
spark.read.format('json').option('inferSchema','true').option('mode','FAILFAST')\
.load('2015-summary.json').show(3)
# csvFile -- json write

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [101]:
csvFile.write.format('json').mode('overwrite').save('my-json-file.json');

In [102]:
spark.read.format('json').option('inferSchema','true').option('mode','FAILFAST')\
.load('my-json-file.json').show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [103]:
# parquet 분석용 데이터를 저장하는데 사용되는 파일 형기
# 압축 / 분할 저장 / 스키마 지원 / 서드파티 지원 / 열 지향 저장

In [104]:
# from pyspark.sql import SparkSession
# spark2 = SparkSession.builder.appName('Create Parquet File').getOrCreate()

In [105]:
data = [
    ('a',10),('b',20),('c',30)
]
df = spark.createDataFrame(data,['name','age'])

In [106]:
df.write.mode('overwrite').parquet('output.parquet')

In [107]:
# spark2.stop()

In [108]:
spark.read.format('parquet').load('output.parquet').show()

+----+---+
|name|age|
+----+---+
|   b| 20|
|   c| 30|
|   a| 10|
+----+---+



In [109]:
csvFile.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [110]:
csvFile.write.format('parquet').mode('overwrite').save('csvFile-parquet')

In [111]:
# ORC(Optimized Row Columnar) file ?
# 대규모 데이터 저장 및 처리 hive 에서 분산처리할때 사용

In [112]:
spark.read.format('orc').load('2010-summary.orc').show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
+-----------------+-------------------+-----+
only showing top 3 rows



In [113]:
# ORC save
csvFile.write.format('orc').mode('overwrite').save('my-orc-file.orc')

In [114]:
spark.read.format('orc').load('my-orc-file.orc').show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [115]:
# mysql connection

In [116]:
mysql_spark = SparkSession.builder.config('spark.jars', "mysql-connector-java-5.1.46.jar")\
.master('local').appName('pySpark_MySql').getOrCreate()

In [117]:
df = (
    mysql_spark.read.format('jdbc')
    .option("url",'jdbc:mysql://localhost:3306/hadoopguide?useSSL=false')
    .option('driver','com.mysql.jdbc.Driver')
    .option('dbtable','tbl_target')
    .option('user','root').option('password','password')
    .load()
)

In [118]:
df.show()

+------+--------+
|testno|testname|
+------+--------+
|     4|   44444|
|     5|   55555|
|     1|   11111|
|     2|   22222|
|     3|   33333|
+------+--------+



In [119]:
driver = "com.mysql.jdbc.Driver"
url = "jdbc:mysql://localhost:3306/hadoopguide?useSSL=false"
tablename = "tbl_target"
user = 'root'
password = 'password'

dbDataFrame = spark.read.format("jdbc")\
    .option("url", url)\
    .option("dbtable", tablename)\
    .option("driver",  driver)\
    .option("user",user)\
    .option('password',password)\
    .load()

In [120]:
dbDataFrame.show()

+------+--------+
|testno|testname|
+------+--------+
|     4|   44444|
|     5|   55555|
|     1|   11111|
|     2|   22222|
|     3|   33333|
+------+--------+



In [121]:
sql_query = '''
    (select * from tbl_target where testno in (1,3,5))
    as myinfo
'''
tablename = sql_query
dbDataFrame = spark.read.format("jdbc")\
    .option("url", url)\
    .option("driver",  driver)\
    .option("user",user)\
    .option('password',password)\
    .option("dbtable", tablename)\
    .load()
dbDataFrame.show()

+------+--------+
|testno|testname|
+------+--------+
|     5|   55555|
|     1|   11111|
|     3|   33333|
+------+--------+



In [122]:
tablename = 'tbl_target'
dbDataFrame = spark.read.format("jdbc")\
    .option("url", url)\
    .option("driver",  driver)\
    .option("user",user)\
    .option('password',password)\
    .option("dbtable", tablename)\
    .option('numPartitions',10)\
    .load()
dbDataFrame.show()

+------+--------+
|testno|testname|
+------+--------+
|     4|   44444|
|     5|   55555|
|     1|   11111|
|     2|   22222|
|     3|   33333|
+------+--------+



In [123]:
# testno !=4 and testname != 55555
predicates = [
    "testno != 4 and testname != '55555'"    
]
props = {'driver':driver,'user':user,'password':password}
spark.read.jdbc(url,tablename,predicates=predicates, properties=props).show()

+------+--------+
|testno|testname|
+------+--------+
|     1|   11111|
|     2|   22222|
|     3|   33333|
+------+--------+



In [124]:
help(spark.read.jdbc)

Help on method jdbc in module pyspark.sql.readwriter:

jdbc(url: str, table: str, column: Optional[str] = None, lowerBound: Union[str, int, NoneType] = None, upperBound: Union[str, int, NoneType] = None, numPartitions: Optional[int] = None, predicates: Optional[List[str]] = None, properties: Optional[Dict[str, str]] = None) -> 'DataFrame' method of pyspark.sql.readwriter.DataFrameReader instance
    Construct a :class:`DataFrame` representing the database table named ``table``
    accessible via JDBC URL ``url`` and connection ``properties``.
    
    Partitions of the table will be retrieved in parallel if either ``column`` or
    ``predicates`` is specified. ``lowerBound``, ``upperBound`` and ``numPartitions``
    is needed when ``column`` is specified.
    
    If both ``column`` and ``predicates`` are specified, ``column`` will be used.
    
    .. versionadded:: 1.4.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    table : 

In [125]:
spark.read.jdbc(url,tablename,predicates=predicates, properties=props)\
.rdd.getNumPartitions()

1

In [126]:
predicates = [
    "testno != 4 and testname != '55555'"    
]
props = {'driver':driver,'user':user,'password':password}
spark.read.jdbc(url,tablename,predicates=predicates, properties=props).count()

3

In [127]:
def test(a,b,c):
    print(a,b,c)

params = {
    'a':1, 'b':2, 'c':3
}
test(1,2,3)

test(**params)

params2 = [1,2,3]
test(*params2)

1 2 3
1 2 3
1 2 3


In [128]:
predicates = [
    "testno != 4 and testname != '55555'"    
]
props = {'driver':driver,'user':user,'password':password}

params = {
    'url': url,
    'table' :  tablename,
    'predicates' : predicates,
    'properties' : props    
}

spark.read.jdbc(**params).show()

+------+--------+
|testno|testname|
+------+--------+
|     1|   11111|
|     2|   22222|
|     3|   33333|
+------+--------+



In [129]:
# mysql table read.... 다양하게 출력
# schema : mysql    table : help_keyword
# 1 read, variable select......

# mysql table read----- [format.... jdbc]


In [130]:
props = {'driver':driver,'user':user,'password':password}

url = "jdbc:mysql://localhost:3306/mysql?useSSL=false"
tablename = 'help_keyword'
params = {
    'url': url,
    'table' :  tablename,    
    'properties' : props    
}

help_keyword_df = spark.read.jdbc(**params)


In [131]:
help_keyword_df.createOrReplaceTempView('help_keyword')

In [132]:
spark.sql("""
    select * from help_keyword limit 5
""").show()

+---------------+--------------------+
|help_keyword_id|                name|
+---------------+--------------------+
|            108|%                ...|
|            264|&                ...|
|            422|(JSON            ...|
|             86|*                ...|
|             84|+                ...|
+---------------+--------------------+



In [133]:
filepath = 'bydata/by-day/2011-10-23.csv'
df = spark.read.format('csv').option("header","true").option('inferSchema','true').load(filepath)
df.count()

1728

In [134]:
help(df.write.jdbc)

Help on method jdbc in module pyspark.sql.readwriter:

jdbc(url: str, table: str, mode: Optional[str] = None, properties: Optional[Dict[str, str]] = None) -> None method of pyspark.sql.readwriter.DataFrameWriter instance
    Saves the content of the :class:`DataFrame` to an external database table via JDBC.
    
    .. versionadded:: 1.4.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    table : str
        Name of the table in the external database.
    mode : str, optional
        specifies the behavior of the save operation when data already exists.
    
        * ``append``: Append contents of this :class:`DataFrame` to existing data.
        * ``overwrite``: Overwrite existing data.
        * ``ignore``: Silently ignore this operation if data already exists.
        * ``error`` or ``errorifexists`` (default case): Throw an exception if data already                 exists.
    properties : dict
        a dictionary of JDBC da

In [135]:
props = {'driver':driver,'user':user,'password':password}

url = "jdbc:mysql://localhost:3306/hadoopguide?useSSL=false"
tablename = 'flight_data'
mode = 'overwrite'
params = {
    'url': url,
    'table' :  tablename,    
    'properties' : props,
    'mode' : mode
}

df.write.jdbc(**params)


In [204]:
filepath = 'bydata/by-day/*.csv'
df = spark.read.format('csv').option("header","true").option('inferSchema','true').load(filepath)
df.count()

541909

In [205]:
props = {'driver':driver,'user':user,'password':password}

url = "jdbc:mysql://localhost:3306/hadoopguide?useSSL=false"
tablename = 'flight_data'
mode = 'overwrite'
partitionColumn = 'InvoiceDate'
params = {
    'url': url,
    'table' :  tablename,    
    'properties' : props,
    'mode' : mode,
    # 'partitionColumn':partitionColumn,
    # 'lowerBound' : 0, 
    # 'upperBound' : 10
}

df.write.jdbc(**params)

In [190]:
df.count()

541909

In [191]:
# CRUD
# Read - select
df.show(1)

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



In [192]:
# Read   Quantity >= 100  df_over_100
predicates = [
    "Quantity >= 100"    
]
props = {'driver':driver,'user':user,'password':password}

params = {
    'url': url,
    'table' :  tablename,
    'predicates' : predicates,
    'properties' : props    
}

df_over_100 = spark.read.jdbc(**params)

In [193]:
df_over_100.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|   571937|    16045|POPART WOODEN PEN...|     100|2011-10-20 09:28:00|     0.04|   14911.0|   EIRE|
|   571937|    22161|HEART DECORATION ...|     240|2011-10-20 09:28:00|     0.19|   14911.0|   EIRE|
|   571937|    22266|EASTER DECORATION...|     640|2011-10-20 09:28:00|     0.19|   14911.0|   EIRE|
|   571937|    22267|EASTER DECORATION...|     120|2011-10-20 09:28:00|     0.39|   14911.0|   EIRE|
|   571937|    22286|DECORATION WOBBLY...|     120|2011-10-20 09:28:00|     0.39|   14911.0|   EIRE|
+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
only showing top 5 rows



In [194]:
# update -- read data and update and overwrite
# not recomment

In [195]:
props = {'driver':driver,'user':user,'password':password}

url = "jdbc:mysql://localhost:3306/hadoopguide?useSSL=false"
tablename = 'flight_data'
params = {
    'url': url,
    'table' :  tablename,    
    'properties' : props,    
}

flight_df = spark.read.jdbc(**params)
flight_df.count()

541909

In [206]:
# spark dataFrame  immutable .... 불변
from pyspark.sql.functions import col
update_df = flight_df.withColumn('UnitPrice',col('UnitPrice')*100)
update_df.show(2)
update_df.count()

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084| RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|    179.0|   14075.0|United Kingdom|
|   580538|    23077|DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|    125.0|   14075.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



541909

In [208]:
[item for item in params.items()]
params['mode'] = 'append'
params['table'] = 'flight_data'
print(params)
update_df.write.jdbc(**params)

{'url': 'jdbc:mysql://localhost:3306/hadoopguide?useSSL=false', 'table': 'flight_data', 'properties': {'driver': 'com.mysql.jdbc.Driver', 'user': 'root', 'password': 'password'}, 'mode': 'append'}


In [198]:
update_df_copy = update_df.select('*')

In [199]:
update_df_copy.count(), update_df.count()

(541909, 541909)

In [200]:
update_df_copy.write.jdbc(**params)

In [202]:
update_df_copy.show(2)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084| RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|    179.0|   14075.0|United Kingdom|
|   580538|    23077|DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|    125.0|   14075.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [203]:
[item for item in params.items()]
params['mode'] = 'overwrite'
params['table'] = 'flight_data'
print(params)
update_df_copy.write.jdbc(**params)

{'url': 'jdbc:mysql://localhost:3306/hadoopguide?useSSL=false', 'table': 'flight_data', 'properties': {'driver': 'com.mysql.jdbc.Driver', 'user': 'root', 'password': 'password'}, 'mode': 'overwrite'}
